### Install Hugging Face Transformers

In [ ]:
!pip3 install transformers==4.4.2

### Download Pre-trained Model and Generate a model.tar.gz file

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
import tarfile

def generate_model_targz_file():
    pretrained = "distilbert-base-uncased-finetuned-sst-2-english"
    pretrained_model = AutoModelForSequenceClassification.from_pretrained(pretrained)
    pretrained_model.save_pretrained(save_directory=".")
    
    tar = tarfile.open("model.tar.gz", "w:gz")
    tar.add("pytorch_model.bin")
    tar.add("config.json")
    tar.close()

In [ ]:
generate_targz_file()

In [ ]:
%%bash

rm pytorch_model.bin
rm config.json

### Upload model.tar.gz file to Amazon S3

In [ ]:
s3_bucket = "<INSERT NEW S3 BUCKET NAME>"
prefix = "pretrained"

In [ ]:
!aws s3 mb s3://{s3_bucket}

In [ ]:
model_data = "s3://{}/{}/model/model.tar.gz".format(s3_bucket, prefix)

In [ ]:
!aws s3 cp model.tar.gz {model_data}

### Deploy pre-trained model to a SageMaker real-time inference endpoint

In [ ]:
from sagemaker import get_execution_role 
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.pytorch.model import PyTorchModel

In [ ]:
execution_role = get_execution_role()
source_dir="scripts"
entry_point = "inference.py"
framework_version = "1.6.0"
endpoint_name = "model-endpoint"
instance_type = "ml.m5.xlarge"

In [ ]:
model = PyTorchModel(
    model_data=model_data, 
    role=execution_role, 
    source_dir=source_dir,
    entry_point=entry_point, 
    framework_version=framework_version,
    py_version="py3"
)

predictor = model.deploy(
    instance_type=instance_type, 
    initial_instance_count=1,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    endpoint_name=endpoint_name
)

### Perform sample predictions

In [ ]:
payload = {
    "text": "I love building penetration testing labs!"
}

predictor.predict(payload)

In [ ]:
from time import sleep

statements = [
    "You're doing an amazing job! Keep up the great work!",
    "Your positive attitude is contagious and inspiring!",
    "You have a unique talent that shines brightly. Keep embracing your strengths and achieving great things!",
    "I understand that things didn't go as planned.",
    "I'm sorry to hear about your disappointment."
]

for statement in statements:
    payload = {
        "text": statement
    }

    result = predictor.predict(payload)
    print(result + ": " + statement)
    sleep(1)

### Transfer ML inference endpoint invoke script to AWS Lambda

In [ ]:
import boto3
import json

sagemaker_client = boto3.client('sagemaker-runtime')


def predict_using_boto3_client(endpoint_name, statement):
    payload = {
        "text": statement
    }

    response = sagemaker_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(payload)
    )

    result = json.loads(response['Body'].read().decode('utf-8'))
    return result

In [ ]:
from time import sleep

statements = [
    "You're doing an amazing job! Keep up the great work!",
    "Your positive attitude is contagious and inspiring!",
    "You have a unique talent that shines brightly. Keep embracing your strengths and achieving great things!",
    "I understand that things didn't go as planned.",
    "I'm sorry to hear about your disappointment."
]

for statement in statements:
    result = predict_using_boto3_client(endpoint_name, statement)
    print(result + ": " + statement)
    sleep(1)

In [ ]:
import boto3

def get_account_id():
    sts_client = boto3.client('sts')
    response = sts_client.get_caller_identity()
    
    return response['Account']

**NOTE**: Here, we use a Lambda execution role with the following IAM managed policies:

- `IAMFullAccess`
- `AmazonSageMakerFullAccess`

In [ ]:
function_name = 'sagemaker-invoker'
role_name = '<LAMBDA ROLE NAME>'
role_arn = f'arn:aws:iam::{get_account_id()}:role/{role_name}'

In [ ]:
import boto3
import io
import zipfile


def create_function(function_name, code, role_arn, handler='lambda_function.handler', runtime='python3.8'):
    lambda_client = boto3.client('lambda')
    
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w') as zf:
        zf.writestr('lambda_function.py', code)

    zip_buffer.seek(0)
    
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler,
        Code={
            'ZipFile': zip_buffer.read()
        }
    )

    return response['FunctionArn']

In [ ]:
lambda_code = '''
import boto3
import json

sagemaker_client = boto3.client('sagemaker-runtime')


def predict_using_boto3_client(endpoint_name, statement):
    payload = {
        "text": statement
    }

    response = sagemaker_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(payload)
    )

    result = json.loads(response['Body'].read().decode('utf-8'))
    return result
    
    
def handler(event, context):
    endpoint_name = event.get('endpoint_name', None)
    statement = event.get('statement', None)
    
    result = predict_using_boto3_client(endpoint_name=endpoint_name,
                                        statement=statement)
    
    return {
        'statusCode': 200,
        'body': result
    }
'''


create_function(function_name=function_name,
                code=lambda_code,
                role_arn=role_arn)

### Invoke the Lambda Function (which invokes a SageMaker endpoint)

In [ ]:
import boto3
import json


def invoke_function(function_name, payload = {}):
    lambda_client = boto3.client('lambda')

    response = lambda_client.invoke(
        FunctionName=function_name,
        Payload=bytes(json.dumps(payload), 'utf-8')
    )

    result = response['Payload'].read().decode('utf-8')
    return result

In [ ]:
payload = {
    'endpoint_name': endpoint_name,
    'statement': "I love building penetration testing labs!"
}

result = invoke_function(function_name, payload=payload)
result

### Cleaning Up

In [ ]:
import boto3
from botocore.exceptions import ClientError


def delete_function(function_name):
    lambda_client = boto3.client('lambda')
    success = False

    try:
        lambda_client.delete_function(FunctionName=function_name)
        success = True
    except ClientError as e:
        print(f"Error deleting Lambda function '{function_name}': {e}")
        
    return success

In [ ]:
delete_function(function_name)

In [ ]:
predictor.delete_endpoint()